In [5]:
import numpy as np 

import pandas as pd 

import json 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 


import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

In [6]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [7]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

## Explore neighborhoods in NYC

In [8]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)


In [9]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [10]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [11]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [12]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [13]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

## Explore the neighborhoods in Queens

In [14]:
queens_data = neighborhoods[neighborhoods['Borough'] == 'Queens'].reset_index(drop=True)
queens_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138


In [39]:
address = 'Queens, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Queens are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Queens are 40.7498243, -73.7976337.


In [16]:
# create map of Queens using latitude and longitude values
map_queens = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(queens_data['Latitude'], queens_data['Longitude'], queens_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_queens)  
    
map_queens

In [18]:
CLIENT_ID = 'YM0FQLFGKJ4XJGG4YNGTZ0AFUVIE3YQVRTBOAJ4I5IBYKKZR'
CLIENT_SECRET = 'URKCO5BBXOIV1W5Y0G2AVU10BKDKELKIAXXN23AUVQW0PZXU'
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YM0FQLFGKJ4XJGG4YNGTZ0AFUVIE3YQVRTBOAJ4I5IBYKKZR
CLIENT_SECRET:URKCO5BBXOIV1W5Y0G2AVU10BKDKELKIAXXN23AUVQW0PZXU


In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
queens_venues = getNearbyVenues(names=queens_data['Neighborhood'],
                                   latitudes=queens_data['Latitude'],
                                   longitudes=queens_data['Longitude']
                                  )

Astoria
Woodside
Jackson Heights
Elmhurst
Howard Beach
Corona
Forest Hills
Kew Gardens
Richmond Hill
Flushing
Long Island City
Sunnyside
East Elmhurst
Maspeth
Ridgewood
Glendale
Rego Park
Woodhaven
Ozone Park
South Ozone Park
College Point
Whitestone
Bayside
Auburndale
Little Neck
Douglaston
Glen Oaks
Bellerose
Kew Gardens Hills
Fresh Meadows
Briarwood
Jamaica Center
Oakland Gardens
Queens Village
Hollis
South Jamaica
St. Albans
Rochdale
Springfield Gardens
Cambria Heights
Rosedale
Far Rockaway
Broad Channel
Breezy Point
Steinway
Beechhurst
Bay Terrace
Edgemere
Arverne
Rockaway Beach
Neponsit
Murray Hill
Floral Park
Holliswood
Jamaica Estates
Queensboro Hill
Hillcrest
Ravenswood
Lindenwood
Laurelton
Lefrak City
Belle Harbor
Rockaway Park
Somerville
Brookville
Bellaire
North Corona
Forest Hills Gardens
Jamaica Hills
Utopia
Pomonok
Astoria Heights
Hunters Point
Sunnyside Gardens
Blissville
Roxbury
Middle Village
Malba
Hammels
Bayswater
Queensbridge


In [21]:
print(queens_venues.shape)
queens_venues.head()

(2051, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Astoria,40.768509,-73.915654,Favela Grill,40.767348,-73.917897,Brazilian Restaurant
1,Astoria,40.768509,-73.915654,Orange Blossom,40.769856,-73.917012,Gourmet Shop
2,Astoria,40.768509,-73.915654,Titan Foods Inc.,40.769198,-73.919253,Gourmet Shop
3,Astoria,40.768509,-73.915654,CrossFit Queens,40.769404,-73.918977,Gym
4,Astoria,40.768509,-73.915654,Off The Hook,40.767200,-73.918104,Seafood Restaurant


In [45]:
print('There are {} uniques categories.'.format(len(queens_venues['Venue Category'].unique())))

There are 268 uniques categories.


In [23]:
# one hot encoding
queens_onehot = pd.get_dummies(queens_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
queens_onehot['Neighborhood'] = queens_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [queens_onehot.columns[-1]] + list(queens_onehot.columns[:-1])
queens_onehot = queens_onehot[fixed_columns]

queens_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
queens_onehot.shape

(2051, 268)

In [25]:
queens_grouped = queens_onehot.groupby('Neighborhood').mean().reset_index()
queens_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,...,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Women's Store
0,Arverne,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.058824,0.000000
1,Astoria,0.000000,0.000000,0.000000,0.010753,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.010753,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.010753,0.000000
2,Astoria Heights,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
3,Auburndale,0.000000,0.000000,0.000000,0.055556,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
4,Bay Terrace,0.000000,0.027027,0.000000,0.054054,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.027027,0.000000,0.000000,0.0,0.027027,0.000000,0.000000,0.054054
5,Bayside,0.013889,0.000000,0.000000,0.041667,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.013889,0.0,0.000000,0.013889,0.000000,0.000000
6,Bayswater,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
7,Beechhurst,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
8,Bellaire,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
9,Belle Harbor,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


In [26]:
queens_grouped.shape

(81, 268)

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Qneighborhoods_venues_sorted = pd.DataFrame(columns=columns)
Qneighborhoods_venues_sorted['Neighborhood'] = queens_grouped['Neighborhood']

for ind in np.arange(queens_grouped.shape[0]):
    Qneighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(queens_grouped.iloc[ind, :], num_top_venues)

Qneighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arverne,Surf Spot,Sandwich Place,Metro Station,Thai Restaurant,Bus Stop,Beach,Coffee Shop,Donut Shop,Board Shop,Playground
1,Astoria,Bar,Middle Eastern Restaurant,Hookah Bar,Greek Restaurant,Deli / Bodega,Indian Restaurant,Mediterranean Restaurant,Seafood Restaurant,Bakery,Food Truck
2,Astoria Heights,Playground,Italian Restaurant,Plaza,Bus Station,Bowling Alley,Supermarket,Liquor Store,Burger Joint,Bakery,Food
3,Auburndale,Miscellaneous Shop,Discount Store,Supermarket,Korean Restaurant,Furniture / Home Store,Fast Food Restaurant,Gymnastics Gym,Toy / Game Store,Athletics & Sports,Train
4,Bay Terrace,Clothing Store,Women's Store,Kids Store,Lingerie Store,Mobile Phone Shop,Shoe Store,American Restaurant,Donut Shop,Cosmetics Shop,Coffee Shop


In [29]:
# set number of clusters
kclusters = 5

queens_grouped_clustering = queens_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(queens_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 0, 1, 1, 1], dtype=int32)

In [30]:
# add clustering labels
Qneighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

queens_merged = queens_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
queens_merged = queens_merged.join(Qneighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
queens_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Queens,Astoria,40.768509,-73.915654,1,Bar,Middle Eastern Restaurant,Hookah Bar,Greek Restaurant,Deli / Bodega,Indian Restaurant,Mediterranean Restaurant,Seafood Restaurant,Bakery,Food Truck
1,Queens,Woodside,40.746349,-73.901842,1,Thai Restaurant,Grocery Store,Filipino Restaurant,Latin American Restaurant,Bakery,American Restaurant,Pub,Donut Shop,Bar,Ice Cream Shop
2,Queens,Jackson Heights,40.751981,-73.882821,1,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Bakery,Mexican Restaurant,Thai Restaurant,Mobile Phone Shop,Pizza Place,Spanish Restaurant,Empanada Restaurant
3,Queens,Elmhurst,40.744049,-73.881656,1,Thai Restaurant,Mexican Restaurant,South American Restaurant,Vietnamese Restaurant,Chinese Restaurant,Bar,Indonesian Restaurant,Sushi Restaurant,Malay Restaurant,Salon / Barbershop
4,Queens,Howard Beach,40.654225,-73.838138,1,Italian Restaurant,Sandwich Place,Pharmacy,Bagel Shop,Fast Food Restaurant,Café,Sporting Goods Shop,Market,Breakfast Spot,Mexican Restaurant


In [31]:
# create map
Qmap_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(queens_merged['Latitude'], queens_merged['Longitude'], queens_merged['Neighborhood'], queens_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(Qmap_clusters)
       
Qmap_clusters

In [32]:
queens_merged.loc[queens_merged['Cluster Labels'] == 0, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Somerville,Park,Women's Store,Fast Food Restaurant,Empanada Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Filipino Restaurant,Egyptian Restaurant
79,Bayswater,Park,Playground,Women's Store,Fast Food Restaurant,Empanada Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Filipino Restaurant


In [33]:
queens_merged.loc[queens_merged['Cluster Labels'] == 1, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Astoria,Bar,Middle Eastern Restaurant,Hookah Bar,Greek Restaurant,Deli / Bodega,Indian Restaurant,Mediterranean Restaurant,Seafood Restaurant,Bakery,Food Truck
1,Woodside,Thai Restaurant,Grocery Store,Filipino Restaurant,Latin American Restaurant,Bakery,American Restaurant,Pub,Donut Shop,Bar,Ice Cream Shop
2,Jackson Heights,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Bakery,Mexican Restaurant,Thai Restaurant,Mobile Phone Shop,Pizza Place,Spanish Restaurant,Empanada Restaurant
3,Elmhurst,Thai Restaurant,Mexican Restaurant,South American Restaurant,Vietnamese Restaurant,Chinese Restaurant,Bar,Indonesian Restaurant,Sushi Restaurant,Malay Restaurant,Salon / Barbershop
4,Howard Beach,Italian Restaurant,Sandwich Place,Pharmacy,Bagel Shop,Fast Food Restaurant,Café,Sporting Goods Shop,Market,Breakfast Spot,Mexican Restaurant
5,Corona,Mexican Restaurant,Sandwich Place,Italian Restaurant,South American Restaurant,Supermarket,Empanada Restaurant,Bakery,Donut Shop,Restaurant,Park
6,Forest Hills,Gym / Fitness Center,Gym,Yoga Studio,Thai Restaurant,Park,Pharmacy,Pizza Place,Convenience Store,Peruvian Restaurant,Donut Shop
7,Kew Gardens,Chinese Restaurant,Indian Restaurant,Bar,Bank,Pet Store,Pharmacy,Cosmetics Shop,Pizza Place,Spa,Pub
8,Richmond Hill,Pizza Place,Lounge,Latin American Restaurant,Bank,Gym / Fitness Center,Sandwich Place,Supermarket,Café,Clothing Store,Moving Target
9,Flushing,Chinese Restaurant,Hotpot Restaurant,Karaoke Bar,Korean Restaurant,Szechuan Restaurant,Bakery,Bubble Tea Shop,Construction & Landscaping,Gym / Fitness Center,Café


In [34]:
queens_merged.loc[queens_merged['Cluster Labels'] == 2, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Neponsit,Beach,Women's Store,Filipino Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,French Restaurant


In [35]:
queens_merged.loc[queens_merged['Cluster Labels'] == 3, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,Brookville,Deli / Bodega,Women's Store,Fish & Chips Shop,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market


In [36]:
queens_merged.loc[queens_merged['Cluster Labels'] == 4, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,Breezy Point,Monument / Landmark,Trail,Bus Stop,Beach,Women's Store,Filipino Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


## Explore neighborhoods in Brooklyn

In [37]:
brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [38]:
address = 'Brooklyn, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.


In [40]:
# create map of Brooklyn using latitude and longitude values
map_brooklyn = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(brooklyn_data['Latitude'], brooklyn_data['Longitude'], brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brooklyn)  
    
map_brooklyn

In [41]:
brooklyn_venues = getNearbyVenues(names=brooklyn_data['Neighborhood'],
                                   latitudes=brooklyn_data['Latitude'],
                                   longitudes=brooklyn_data['Longitude']
                                  )


Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


In [42]:
print(brooklyn_venues.shape)
brooklyn_venues.head()

(2711, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa
1,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop
2,Bay Ridge,40.625801,-74.030621,Leo's Casa Calamari,40.624200,-74.030931,Pizza Place
3,Bay Ridge,40.625801,-74.030621,Pegasus Cafe,40.623168,-74.031186,Breakfast Spot
4,Bay Ridge,40.625801,-74.030621,The Bookmark Shoppe,40.624577,-74.030562,Bookstore


In [44]:
print('There are {} uniques categories.'.format(len(brooklyn_venues['Venue Category'].unique())))

There are 290 uniques categories.


In [46]:
# one hot encoding
brooklyn_onehot = pd.get_dummies(brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
brooklyn_onehot['Neighborhood'] = brooklyn_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]

brooklyn_onehot.head()

,Yoga Studio,Accessories Store,American Restaurant,Animal Shelter,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
brooklyn_onehot.shape

(2711, 290)

In [48]:
brooklyn_grouped = brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
brooklyn_grouped

,Neighborhood,Yoga Studio,Accessories Store,American Restaurant,Animal Shelter,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Bath Beach,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.021277,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,Bay Ridge,0.000000,0.000000,0.032258,0.00,0.000000,0.00,0.000000,0.000000,0.010753,...,0.000000,0.010753,0.000000,0.010753,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,Bedford Stuyvesant,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.035714,0.00,0.000000
3,Bensonhurst,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,Bergen Beach,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,Boerum Hill,0.021277,0.000000,0.000000,0.00,0.010638,0.00,0.000000,0.010638,0.031915,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010638,0.00,0.000000
6,Borough Park,0.000000,0.000000,0.045455,0.00,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
7,Brighton Beach,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
8,Broadway Junction,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,Brooklyn Heights,0.040000,0.000000,0.020000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,0.020000,0.00,0.000000


In [49]:
brooklyn_grouped.shape

(70, 290)

In [51]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Bneighborhoods_venues_sorted = pd.DataFrame(columns=columns)
Bneighborhoods_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighborhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    Bneighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

Bneighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Chinese Restaurant,Pharmacy,Gas Station,Bubble Tea Shop,Fast Food Restaurant,Italian Restaurant,Sushi Restaurant,Check Cashing Service,Surf Spot,Bike Rental / Bike Share
1,Bay Ridge,Spa,Italian Restaurant,Pizza Place,Grocery Store,American Restaurant,Greek Restaurant,Chinese Restaurant,Bar,Ice Cream Shop,Bagel Shop
2,Bedford Stuyvesant,Deli / Bodega,Coffee Shop,Bar,Café,Pizza Place,Tiki Bar,New American Restaurant,Gift Shop,Gourmet Shop,Juice Bar
3,Bensonhurst,Chinese Restaurant,Ice Cream Shop,Italian Restaurant,Donut Shop,Sushi Restaurant,Pizza Place,Bridal Shop,Coffee Shop,Shabu-Shabu Restaurant,Flower Shop
4,Bergen Beach,Harbor / Marina,Athletics & Sports,Other Repair Shop,Playground,Donut Shop,Baseball Field,Women's Store,Factory,Falafel Restaurant,Farm


In [52]:
# set number of clusters
kclusters = 5

brooklyn_grouped_clustering = brooklyn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brooklyn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 0, 2, 0, 0, 2, 0, 0, 0], dtype=int32)

In [53]:
# add clustering labels
Bneighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

brooklyn_merged =brooklyn_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
brooklyn_merged = brooklyn_merged.join(Bneighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

brooklyn_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,0,Spa,Italian Restaurant,Pizza Place,Grocery Store,American Restaurant,Greek Restaurant,Chinese Restaurant,Bar,Ice Cream Shop,Bagel Shop
1,Brooklyn,Bensonhurst,40.611009,-73.995180,2,Chinese Restaurant,Ice Cream Shop,Italian Restaurant,Donut Shop,Sushi Restaurant,Pizza Place,Bridal Shop,Coffee Shop,Shabu-Shabu Restaurant,Flower Shop
2,Brooklyn,Sunset Park,40.645103,-74.010316,2,Latin American Restaurant,Bakery,Bank,Pizza Place,Mexican Restaurant,Pharmacy,Deli / Bodega,Fried Chicken Joint,Mobile Phone Shop,Gym
3,Brooklyn,Greenpoint,40.730201,-73.954241,0,Bar,Pizza Place,Coffee Shop,Cocktail Bar,Yoga Studio,Grocery Store,Thrift / Vintage Store,Sushi Restaurant,Bakery,Café
4,Brooklyn,Gravesend,40.595260,-73.973471,2,Pizza Place,Bakery,Chinese Restaurant,Bus Station,Martial Arts Dojo,Lounge,Hookah Bar,Diner,Pharmacy,Donut Shop


In [55]:
# create map
Bmap_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(brooklyn_merged['Latitude'], brooklyn_merged['Longitude'], brooklyn_merged['Neighborhood'], brooklyn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(Bmap_clusters)
       
Bmap_clusters

In [57]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 0, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Ridge,Spa,Italian Restaurant,Pizza Place,Grocery Store,American Restaurant,Greek Restaurant,Chinese Restaurant,Bar,Ice Cream Shop,Bagel Shop
3,Greenpoint,Bar,Pizza Place,Coffee Shop,Cocktail Bar,Yoga Studio,Grocery Store,Thrift / Vintage Store,Sushi Restaurant,Bakery,Café
5,Brighton Beach,Beach,Restaurant,Russian Restaurant,Eastern European Restaurant,Sushi Restaurant,Gourmet Shop,Mobile Phone Shop,Bank,Bakery,Taco Place
6,Sheepshead Bay,Dessert Shop,Turkish Restaurant,Sandwich Place,Yoga Studio,Grocery Store,Creperie,Diner,Restaurant,Outlet Store,Chinese Restaurant
11,Kensington,Thai Restaurant,Grocery Store,Ice Cream Shop,Pizza Place,Furniture / Home Store,Sandwich Place,Lingerie Store,Liquor Store,Spa,Deli / Bodega
12,Windsor Terrace,Grocery Store,Plaza,Park,Café,Deli / Bodega,Diner,Antique Shop,Chinese Restaurant,Sushi Restaurant,Bookstore
13,Prospect Heights,Bar,Mexican Restaurant,Café,Thai Restaurant,Cocktail Bar,Beer Bar,Coffee Shop,Pizza Place,Ice Cream Shop,Bakery
15,Williamsburg,Coffee Shop,Bagel Shop,Bar,Yoga Studio,Food Truck,Korean Restaurant,Taco Place,Latin American Restaurant,Steakhouse,Liquor Store
16,Bushwick,Bar,Deli / Bodega,Coffee Shop,Mexican Restaurant,Bakery,Pizza Place,Thrift / Vintage Store,Discount Store,Vegetarian / Vegan Restaurant,Café
17,Bedford Stuyvesant,Deli / Bodega,Coffee Shop,Bar,Café,Pizza Place,Tiki Bar,New American Restaurant,Gift Shop,Gourmet Shop,Juice Bar


In [58]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 1, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,Paerdegat Basin,Asian Restaurant,Moving Target,Food,Auto Garage,Harbor / Marina,Women's Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


In [59]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 2, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Bensonhurst,Chinese Restaurant,Ice Cream Shop,Italian Restaurant,Donut Shop,Sushi Restaurant,Pizza Place,Bridal Shop,Coffee Shop,Shabu-Shabu Restaurant,Flower Shop
2,Sunset Park,Latin American Restaurant,Bakery,Bank,Pizza Place,Mexican Restaurant,Pharmacy,Deli / Bodega,Fried Chicken Joint,Mobile Phone Shop,Gym
4,Gravesend,Pizza Place,Bakery,Chinese Restaurant,Bus Station,Martial Arts Dojo,Lounge,Hookah Bar,Diner,Pharmacy,Donut Shop
7,Manhattan Terrace,Pizza Place,Donut Shop,Ice Cream Shop,Bakery,Coffee Shop,Mobile Phone Shop,Steakhouse,Organic Grocery,Grocery Store,Bank
9,Crown Heights,Pizza Place,Café,Museum,Candy Store,Grocery Store,Bakery,Supermarket,Salon / Barbershop,Coffee Shop,Deli / Bodega
14,Brownsville,Moving Target,Chinese Restaurant,Park,Restaurant,Pool,Fried Chicken Joint,Trail,Performing Arts Venue,Discount Store,Spanish Restaurant
26,East New York,Deli / Bodega,Spanish Restaurant,Gym,Fried Chicken Joint,Metro Station,Bus Station,Fast Food Restaurant,Event Service,Caribbean Restaurant,Chinese Restaurant
27,Starrett City,Moving Target,Intersection,Chinese Restaurant,Gym Pool,Donut Shop,River,Bus Stop,Pharmacy,Pizza Place,American Restaurant
33,Bath Beach,Chinese Restaurant,Pharmacy,Gas Station,Bubble Tea Shop,Fast Food Restaurant,Italian Restaurant,Sushi Restaurant,Check Cashing Service,Surf Spot,Bike Rental / Bike Share
34,Borough Park,Bank,Pizza Place,Grocery Store,Pharmacy,Hostel,Hotel,Restaurant,Coffee Shop,Chinese Restaurant,Fast Food Restaurant


In [60]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 3, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Mill Island,Other Repair Shop,Pool,Women's Store,Field,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant


In [61]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 4, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Flatbush,Coffee Shop,Deli / Bodega,Mexican Restaurant,Caribbean Restaurant,Bank,Plaza,Middle Eastern Restaurant,Lounge,Liquor Store,Sandwich Place
10,East Flatbush,Pharmacy,Wine Shop,Print Shop,Caribbean Restaurant,Department Store,Food & Drink Shop,Hardware Store,Supermarket,Chinese Restaurant,Fast Food Restaurant
28,Canarsie,Gym,Asian Restaurant,Chinese Restaurant,Caribbean Restaurant,Food,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Event Service
29,Flatlands,Pharmacy,Caribbean Restaurant,Fried Chicken Joint,Fast Food Restaurant,Nightclub,Paper / Office Supplies Store,Bar,Discount Store,Video Store,Lounge
32,Coney Island,Baseball Stadium,Caribbean Restaurant,Pizza Place,Skating Rink,Monument / Landmark,Beach,Theme Park Ride / Attraction,Beer Garden,Food Court,Pharmacy
47,Prospect Park South,Pizza Place,Caribbean Restaurant,Mobile Phone Shop,Grocery Store,Fast Food Restaurant,Latin American Restaurant,Donut Shop,Clothing Store,Fried Chicken Joint,Mexican Restaurant
54,Ditmas Park,Deli / Bodega,Caribbean Restaurant,Chinese Restaurant,Pizza Place,Pharmacy,Burger Joint,Donut Shop,Health Food Store,Japanese Restaurant,Kids Store
56,Rugby,Caribbean Restaurant,Bank,Grocery Store,Fast Food Restaurant,Fried Chicken Joint,Beach,Food,Seafood Restaurant,Chinese Restaurant,Supermarket
57,Remsen Village,Caribbean Restaurant,Fast Food Restaurant,Breakfast Spot,Auto Dealership,Fried Chicken Joint,Fish Market,Supermarket,Farmers Market,Café,Sandwich Place
69,Erasmus,Caribbean Restaurant,Yoga Studio,Music Venue,School,Convenience Store,Playground,Pizza Place,Pharmacy,Donut Shop,Mobile Phone Shop
